# Проект-3. Использование EDA для предсказания рейтинга отелей.

In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [55]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыве;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.
* reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
* total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
* total_number_of_reviews — общее количество действительных отзывов об отеле;
* tags — теги, которые рецензент дал отелю;
* days_since_review — количество дней между датой проверки и датой очистки;
* additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat — географическая широта отеля;
* lng — географическая долгота отеля.

Посмотрим на тип данных.

In [56]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

Пропущенные значения в признаках.

In [57]:
hotels.isnull().sum().sort_values(ascending=False)

lng                                           2448
lat                                           2448
positive_review                                  0
days_since_review                                0
tags                                             0
reviewer_score                                   0
total_number_of_reviews_reviewer_has_given       0
review_total_positive_word_counts                0
hotel_address                                    0
additional_number_of_scoring                     0
review_total_negative_word_counts                0
negative_review                                  0
reviewer_nationality                             0
hotel_name                                       0
average_score                                    0
review_date                                      0
total_number_of_reviews                          0
dtype: int64

Выделим целевой признак в отдельную переменную.

In [58]:
TARGET_FEATURE = hotels['reviewer_score']

## Добавим признак страны отеля и информацию о населении в стране каждого отеля

Посмотрим на пример строк из признака hotel_address, чтобы выделить оттуда важную информацию и увидеть структуру строк.

In [59]:
for i in range(0, 10, 2):
    display(hotels['hotel_address'].iloc[i])

'Stratton Street Mayfair Westminster Borough London W1J 8LT United Kingdom'

'151 bis Rue de Rennes 6th arr 75006 Paris France'

'Molenwerf 1 1014 AG Amsterdam Netherlands'

'97 Cromwell Road Kensington and Chelsea London SW7 4DN United Kingdom'

'190 Queen s Gate Kensington and Chelsea London SW7 5EX United Kingdom'

Возьмем таблицу с Википедии о названии всех стран и соотношение населения к общему населению планеты.

In [60]:
countries_info = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population')[1]
total_population = countries_info['Population']['Numbers'].iloc[0]
countries_info['% of the world'] = countries_info['Population']['Numbers'] / total_population
countries_info = countries_info.sort_index(axis=1).drop(
    columns=['Date', 'Notes', 'Source (official or from the\xa0United Nations)', 'Rank', 'Population'],
    index=0).droplevel(level=1, axis=1).rename(
    columns={'% of the world': 'population_ratio',
             'Country / Dependency': 'country'}).sort_index(axis=1)
countries_info.head()

,country,population_ratio
1,China,0.176189
2,India,0.171676
3,United States,0.041732
4,Indonesia,0.034417
5,Pakistan,0.029431


Создадим признак *country*, в котором будет информация о стране, в которой располагается отель.

In [61]:
def get_country(address:str) -> str:
    
    """Returns existing country for raw address. If the country was not found, then returns none 

    Args:
        address (str): Raw address

    Returns:
        str: One of the existing countries
    """
    
    for country in countries_info['country'].to_list():
        if country in address:
            return country

hotels['country'] = hotels['hotel_address'].apply(get_country)
hotels = hotels.join(countries_info.set_index('country'), on='country')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,country,population_ratio
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,United Kingdom,0.008365
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,United Kingdom,0.008365
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,France,0.008492


## Заполняем пропущенные значения координат с помощью `geopy`

Посмотрим, в каких странах есть пропущенные значения.

In [62]:
countries_with_null = hotels[hotels['lat'].isna() | hotels['lng'].isna()]['country'].value_counts()
countries_with_null

Austria    1990
France      299
Spain       159
Name: country, dtype: int64

Можно заметить, что больше всего пропущенных значений находится в Австрии, а также есть во Франции и Испании.

Посмотрим на структуру адресов в этих странах, чтобы было проще заполнять пустые значения.

In [63]:
for country in countries_with_null.index:
    display(hotels[(hotels['lat'].isna() | hotels['lng'].isna()) 
                   & (hotels['country'] == country)].iloc[0].loc['hotel_address'])

'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria'

'23 Rue Damr mont 18th arr 75018 Paris France'

'Bail n 4 6 Eixample 08010 Barcelona Spain'

Видно, что последние 3 значения в строке представляют собой структуру *[postal code + city + country]*. Можем воспользоваться этой особенностью и заполнить пропущенные координаты с помощью библиотеки `geopy`.

Создадим отдельный датафрейм, который будет содержать в себе адрес, широту и долготу. Пропущенные значения заменим на строку, чтобы можно было заполнить эти значения с помощью метода `apply`, т.к. этот метод не работает с пропущенными (`NaN/None`) значениями.

In [64]:
address_coords = hotels[['hotel_address', 'lat', 'lng']].copy()
address_coords.fillna('none', inplace=True)
address_coords.head()

,hotel_address,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,48.888697,2.39454
4,Molenwerf 1 1014 AG Amsterdam Netherlands,52.385601,4.84706


Отобразим, как выглядят на данном этапе наши пропущенные значения, а потом сравним их после преобразования.

In [65]:
null_indexes = address_coords[address_coords['lat'] == 'none'].index
address_coords.iloc[null_indexes]

,hotel_address,lat,lng
122,Savoyenstra e 2 16 Ottakring 1160 Vienna Austria,none,none
566,23 Rue Damr mont 18th arr 75018 Paris France,none,none
724,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,none,none
754,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,none,none
1137,4 rue de la P pini re 8th arr 75008 Paris France,none,none
...,...,...,...
386092,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,none,none
386465,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,none,none
386504,4 rue de la P pini re 8th arr 75008 Paris France,none,none
386702,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,none,none


Посмотрим на количество уникальных адресов.

In [66]:
address_coords.iloc[null_indexes]['hotel_address'].nunique()

17

In [67]:
# Создадим экземпляр класса Nominatim библиотеки geopy
geolocator = Nominatim(user_agent='project_3')
# Создадим экземпляр класса RateLimiter, чтобы ограничить задержку между запросами. В данном случае выполнение ячейки займет ~17 секунд
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_coords(address:str) -> tuple:
    
    """Returns the latitude and longitude for the address

    Args:
        address (str): Address

    Returns:
        tuple: (latitude, longitude)
    """
    
    # Сократим адрес до формата {почтовый индекс + город + страна}, т.к. в "сыром" формате библиотека geopy может не распознать местоположение
    def get_short_address(address:str) -> str:
        
        """Returns address in format '%postal_code %city %country'
        
        Args:
            address (str): Raw format of address, where last 3 words is '%postal_code %city %country'
            
        Returns:
            str: Short address
        """
        
        short_address = ' '.join(address.split()[-3:])
        return short_address
    
    location = geocode(get_short_address(address))
    lat = location.latitude
    lng = location.longitude
    
    return lat, lng

unique_null_addresses = address_coords.iloc[null_indexes]['hotel_address'].unique()
# Создадим словарь, где ключем будет полный адрес, а значением - кортеж из широты и долготы
filled_addresses_dict = dict()
for address in unique_null_addresses:
    filled_addresses_dict[address] = get_coords(address)

In [68]:
def fill_coords(address:str, lat, lng, dictionary:dict) -> tuple:
    
    """Returns the latitude and longitude value for the address key. It is recommended to use in combination with the pandas.apply() method

    Args:
        address (str): Address
        lat (any): Empty latitude column to fill
        lng (any): Empty longitude column to fill
        dictionary (dict): Dictionary to fill in latitude and longitude 

    Returns:
        tuple: (address, new latitude, new longitude)
    """
    
    lat = dictionary[address][0]
    lng = dictionary[address][1]
    
    return address, lat, lng

address_coords.iloc[null_indexes] = address_coords.iloc[null_indexes].apply(
    lambda row: fill_coords(row['hotel_address'], row['lat'], row['lng'], filled_addresses_dict),
    axis=1,
    result_type='broadcast'
)

Выведем строки, которые были пустыми, после того, как получили их координаты.

In [69]:
address_coords.iloc[null_indexes]

,hotel_address,lat,lng
122,Savoyenstra e 2 16 Ottakring 1160 Vienna Austria,48.212692,16.311863
566,23 Rue Damr mont 18th arr 75018 Paris France,48.889485,2.342177
724,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,48.211029,16.347425
754,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,48.222757,16.356334
1137,4 rue de la P pini re 8th arr 75008 Paris France,48.873751,2.314978
...,...,...,...
386092,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,48.214453,16.397042
386465,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,48.214453,16.397042
386504,4 rue de la P pini re 8th arr 75008 Paris France,48.873751,2.314978
386702,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,48.214453,16.397042


Заменим пустые значения координат в нашем основном датафрейме на преобразованные. Теперь у нас нет пропусков в данных.

In [70]:
filled_coords = address_coords[['lat', 'lng']].copy()
hotels[['lat', 'lng']] = filled_coords
hotels.isnull().sum()

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
reviewer_score                                0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
country                                       0
population_ratio                              0
dtype: int64

Теперь нам больше не нужен признак с полным адресом отеля.

In [71]:
hotels.drop(columns='hotel_address', inplace=True)
hotels.head(3)

,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,country,population_ratio
0,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,United Kingdom,0.008365
1,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,United Kingdom,0.008365
2,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,France,0.008492


## Добавим признак с информацией о столице страны, в которой находится отель.

Возьмем таблицу столиц с Википедии. Добавим для каждой столицы широту и долготу.

In [72]:
capitals_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals')[1]
capitals_df.drop(columns='Notes', axis=1, inplace=True)
capitals_df = capitals_df[capitals_df['Country/Territory'].isin(hotels['country'].unique())]
capitals_df = capitals_df.rename(columns={
    'City/Town': 'capital',
    'Country/Territory': 'country'
}).drop(index=13) # Удалим неофициальную столицу Нидерландов
capitals_df[['lat_cap', 'lng_cap']] = 'none'
capitals_df['capital'] = capitals_df['capital'].apply(lambda x: x.split()[0])
capitals_df.loc[150]['capital'] = 'Mexico city' # Название города совпадает с названием страны, добавим уточнение для geopy, что это является городом
capitals_df

,capital,country,lat_cap,lng_cap
12,Amsterdam,Netherlands,none,none
134,London,United Kingdom,none,none
138,Madrid,Spain,none,none
150,Mexico city,Mexico,none,none
180,Paris,France,none,none
199,Rome,Italy,none,none
248,Vienna,Austria,none,none


In [73]:
def get_capital_coords(address:str) -> tuple:
    
    """Returns the latitude and longitude for the city

    Args:
        address (str): City name

    Returns:
        tuple: (latitude, longitude)
    """
    
    location = geocode(address)
    lat = location.latitude
    lng = location.longitude
    
    return lat, lng

capital_coords = dict()
for capital in capitals_df['capital'].values:
    capital_coords[capital] = get_capital_coords(capital)

capitals_df[['capital', 'lat_cap', 'lng_cap']] = capitals_df[['capital', 'lat_cap', 'lng_cap']].apply(
    lambda row: fill_coords(row['capital'], row['lat_cap'], row['lng_cap'], capital_coords),
    axis=1,
    result_type='broadcast'
)
capitals_df

,capital,country,lat_cap,lng_cap
12,Amsterdam,Netherlands,52.37308,4.892453
134,London,United Kingdom,51.507322,-0.127647
138,Madrid,Spain,40.416705,-3.703582
150,Mexico city,Mexico,19.43263,-99.133178
180,Paris,France,48.85889,2.320041
199,Rome,Italy,41.89332,12.482932
248,Vienna,Austria,48.208354,16.372504


Присоединим к основному датафрейму данные о столицах.

In [76]:
hotels = hotels.join(capitals_df.set_index('country'), on='country')
hotels.head(3)

,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,reviewer_score,tags,days_since_review,lat,lng,country,population_ratio,capital,lat_cap,lng_cap
0,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,...,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,United Kingdom,0.008365,London,51.507322,-0.127647
1,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,...,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,United Kingdom,0.008365,London,51.507322,-0.127647
2,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,...,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,France,0.008492,Paris,48.85889,2.320041


## Посчитаем расстояние между расположением отеля и столицей страны